In [ ]:
!pip install mlflow

In [ ]:
import mlflow
mlflow.set_tracking_uri("file:///content/mlruns")

In [ ]:

# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='file:///content/mlruns/341596156009867538', creation_time=1751426711595, experiment_id='341596156009867538', last_update_time=1751426711595, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [ ]:
!pip install optuna

In [ ]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
df = pd.read_csv('/content/reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [ ]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()


[I 2025-07-02 03:39:47,793] A new study created in memory with name: no-name-68d4ffe0-50c8-4ab8-8270-e2b86a2aa5a8
[I 2025-07-02 03:41:54,591] Trial 0 finished with value: 0.7969382368467359 and parameters: {'n_estimators': 198, 'learning_rate': 0.009259980611910346, 'max_depth': 10}. Best is trial 0 with value: 0.7969382368467359.
[I 2025-07-02 03:43:24,524] Trial 1 finished with value: 0.7288404011965511 and parameters: {'n_estimators': 186, 'learning_rate': 0.0015398220713895587, 'max_depth': 8}. Best is trial 0 with value: 0.7969382368467359.
[I 2025-07-02 03:44:08,058] Trial 2 finished with value: 0.7976420904451874 and parameters: {'n_estimators': 96, 'learning_rate': 0.02621381998075024, 'max_depth': 8}. Best is trial 2 with value: 0.7976420904451874.
[I 2025-07-02 03:44:45,991] Trial 3 finished with value: 0.8720746084814358 and parameters: {'n_estimators': 214, 'learning_rate': 0.06871074243768815, 'max_depth': 6}. Best is trial 3 with value: 0.8720746084814358.
[I 2025-07-02 0

In [ ]:
# Step 1: Install MLflow and pyngrok
!pip install mlflow pyngrok --quiet

# Step 2: Import and set Ngrok token
from pyngrok import ngrok
ngrok.set_auth_token("2zGeN1KVeIEoBe6V3Bju83fWaOY_3m5hd6eue8oSWwytmX6Yr")  # Your token
#Start MLflow UI (default at port 5000)
get_ipython().system_raw("mlflow ui --backend-store-uri file:///content/mlruns --port 5000 &")

#Create public link to access the MLflow UI
public_url = ngrok.connect(5000)
print(f"✅ MLflow UI is live here: {public_url}")